In [1]:
from init import *
import time

# Hull-White model
https://en.wikipedia.org/wiki/Hull%E2%80%93White_model

In [2]:
model = sdk.Model()
model.TimeStart = 0
TimeEnd = 3
model.TimeSteps = 200
model.NumPaths = 10000

model.Add(sdk.IndependentGaussian())

ref_time = -1
points = 10
alpha_const = 1
sigma_const = 0.3

# theta(t) is the expected rate from today
theta_func = lambda t: (0.1 - t/2 + (t/2)**2)
alpha_func = lambda t: alpha_const
sigma_func = lambda t: sigma_const

theta_process = sdk.Linear1DInterpolation(
    theta_func,
    ref_time,
    model.TimeStart,
    TimeEnd,
    points=points,
    title='theta'
)
model.Add(theta_process)

alpha_process = sdk.Linear1DInterpolation(
    alpha_func,
    ref_time,
    model.TimeStart,
    TimeEnd,
    points=points,
    title='alpha'
)
model.Add(alpha_process)

sigma_process = sdk.Linear1DInterpolation(
    sigma_func,
    ref_time,
    model.TimeStart,
    TimeEnd,
    points=points,
    title='sigma'
)
model.Add(sigma_process)

hw_process = Updater(name  = 'HullWhiteModel',
    start = 0,
    args = [],
    refs = [
        theta_process.GetStateNumber(),
        alpha_process.GetStateNumber(),
        sigma_process.GetStateNumber()
    ],
    _title='HW'
)
model.Add(hw_process)

# Zero Coupon Bond
zcb_process = Updater(name  = 'ZeroCouponBond',
    start = 1,
    args = [],
    refs = [hw_process.GetStateNumber()],
    _title='ZCB'
)
model.Add(zcb_process)


print('processes:',[p.GetStateNumber() for p in [theta_process,alpha_process,sigma_process,hw_process]])

# model.AddEvaluationRequest (TimeEnd/2)
model.AddEvaluationRequest (TimeEnd)

print(model.json())

TypeError: Linear1DInterpolation.__init__() got an unexpected keyword argument 'points'

In [ ]:
reply = requests.post(f'{server}/model',model.json())
reply.json()

In [ ]:
er = EvaluationResults(reply.json(),model)
er.df()

In [ ]:
import scipy.integrate
scipy.integrate.quad(sigma_func,model.TimeStart,TimeEnd)

# Constant $\alpha$, $\sigma$ and time-dependent $\theta(t)$ 
$$ r(t)\sim N\left(e^{-\alpha t} r(0) + \int_0^t e^{\alpha(s-t)}\theta(s)\textrm{d}s, \frac{\sigma^2}{2\alpha}\left(1-e^{-2\alpha t} \right) \right) $$

In [ ]:
# the rate seen today (at t=0) is 10% per year
r0 = theta_func(0)
print(f'r(0)={r0}')

In [ ]:
import plotly.express as px
state = 0

vx = linspace(0,TimeEnd,50)

fig = px.scatter (
    title = f'$\\theta(t)$',
    x = vx,
    y = [theta_func(x) for x in vx]
)
fig.show()

In [ ]:
from math import exp, sqrt
import scipy.integrate as integrate
t = TimeEnd
result = integrate.quad(lambda s: exp(alpha_const*(s-t))*theta_func(s), 0,t)
rt_mean = exp(-alpha_const*t)*r0 + result[0]
rt_variance = sigma_const**2/(2*alpha_const) * (1-exp(-2*alpha_const*t))
rt_mean, sqrt(rt_variance)

In [ ]:
er.df()

In [ ]:
er.GetStateEvaluationResult(3,0).mean, er.GetStateEvaluationResult(3,0).MeanError()

In [ ]:
r = er.GetStateEvaluationResult(3,0)
pd.DataFrame([{
    'Method':'Theory','Mean':rt_mean,'Variance':rt_variance
    },{
    'Method':'MC','Mean':r.mean,'Variance':r.stddev**2
}])

In [ ]:
er.df()